In [49]:
import pandas as pd
import os
import numpy as np
import random

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
import sklearn.metrics as metrics
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [51]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

In [52]:
data = pd.read_csv("C:/Users/kt826/Desktop/train월별통계량.csv")
data

,ID,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,...,11월_새순_var,9월_엽록소_mean,9월_엽록소_std,9월_엽록소_var,10월_엽록소_mean,10월_엽록소_std,10월_엽록소_var,11월_엽록소_mean,11월_엽록소_std,11월_엽록소_var
0,TRAIN_0000,692,275.0,287.0,292.0,289.5,2.8,2.8,2.7,2.7,...,0.035066,71.941171,19.395611,376.189722,74.216190,0.839849,0.705346,71.500688,0.788738,0.622108
1,TRAIN_0001,534,293.0,284.0,336.0,310.0,3.3,3.3,3.3,3.2,...,0.132209,73.183501,19.917938,396.724236,74.780315,1.082409,1.171609,72.033788,0.844849,0.713770
2,TRAIN_0002,634,300.0,392.0,450.0,421.0,3.0,2.9,2.9,2.9,...,0.053373,72.503518,19.580020,383.377199,74.796778,0.752497,0.566251,72.045302,0.907441,0.823449
3,TRAIN_0003,639,289.0,368.0,379.0,373.5,3.1,3.0,3.0,3.0,...,0.048505,71.580823,19.283019,371.834825,73.683379,1.021671,1.043811,70.608235,0.782000,0.611525
4,TRAIN_0004,496,306.0,353.0,358.0,355.5,3.7,3.6,3.6,3.6,...,0.083862,69.994498,18.877361,356.354769,72.015112,0.903184,0.815740,68.851328,0.727619,0.529430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,TRAIN_2202,119,230.0,353.0,367.0,360.0,4.7,4.6,4.5,4.5,...,0.157288,67.918006,18.477816,341.429685,69.876305,0.879335,0.773230,67.120400,0.824625,0.680007
2203,TRAIN_2203,611,266.0,337.0,400.0,368.5,3.1,3.0,3.0,3.0,...,0.049987,75.134885,20.266756,410.741415,77.330196,0.806638,0.650665,74.301864,0.900826,0.811488
2204,TRAIN_2204,650,237.0,353.0,362.0,357.5,3.0,3.0,2.9,2.9,...,0.052315,69.271803,18.699799,349.682472,71.372888,0.818776,0.670393,68.254110,0.931277,0.867277
2205,TRAIN_2205,363,281.0,337.0,403.0,370.0,3.9,3.9,3.8,3.8,...,0.090093,68.003378,18.428637,339.614676,70.093678,0.806787,0.650905,67.410442,0.801842,0.642950


In [53]:
from sklearn.model_selection import train_test_split

x = data.drop(['ID','착과량(int)'], axis=1)
y = data['착과량(int)'].values.reshape(-1,1)

x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=42)

In [54]:
def modelfit(pip_xgb, grid_param_xgb, x, y):
    gs_xgb = (GridSearchCV(estimator=pip_xgb,
             param_grid=grid_param_xgb,
                          cv=4,
                          scoring='neg_mean_absolute_error',
                          n_jobs=-1,
                          verbose=10))
    gs_xgb = gs_xgb.fit(x,y)
    print('Train Done')
    
    y_pred = gs_xgb.predict(x)
    
    print("\nModel Report")
    print("\nCV 결과 : ", gs_xgb.cv_results_)
    print("\n베스트 정답률 : ", gs_xgb.best_score_)
    print("\n베스트 파라미터 : ", gs_xgb.best_params_)

In [55]:
pip_xgb1 = Pipeline([('scl', StandardScaler()),
                   ('reg', MultiOutputRegressor(XGBRFRegressor()))])
grid_param_xgb1 = {
    'reg__estimator__max_depth' : [3, 4, 5, 6, 7],
    'reg__estimator__gamma' : [1, 0.1, 0.01, 0.001, 0.0001, 0],
    'reg__estimator__learning_rate' : [0.01, 0.03, 0.05, 0.07, 0.08],
    'reg__estimator__subsample' : [0.4, 0.6, 0.8],
    'reg__estimator__colsample_bytree' : [0.2, 0.6, 0.8]
}

In [56]:
modelfit(pip_xgb1, grid_param_xgb1, x_train, y_train)

Fitting 4 folds for each of 1350 candidates, totalling 5400 fits
Train Done

Model Report

CV 결과 :  {'mean_fit_time': array([ 0.50585997,  0.50996596,  0.48878121, ..., 11.44636369,
       11.54684389,  8.98293561]), 'std_fit_time': array([0.00870502, 0.00277264, 0.010393  , ..., 0.10022595, 0.10960038,
       0.18162922]), 'mean_score_time': array([0.00721014, 0.00853944, 0.0091452 , ..., 0.01525623, 0.01589733,
       0.00391442]), 'std_score_time': array([0.00786566, 0.00424544, 0.00567474, ..., 0.00119168, 0.00244486,
       0.00677997]), 'param_reg__estimator__colsample_bytree': masked_array(data=[0.2, 0.2, 0.2, ..., 0.8, 0.8, 0.8],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_reg__estimator__gamma': masked_array(data=[1, 1, 1, ..., 0, 0, 0],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_reg__estimator__learning_rate': masked_

In [26]:
best_params = {
    'reg__estimator__colsample_bytree': 0.6,
    'reg__estimator__gamma': 1,
    'reg__estimator__learning_rate': 0.05,
    'reg__estimator__max_depth': 3,
    'reg__estimator__subsample': 0.6
}

In [57]:
xgb_model = XGBRFRegressor(
    colsample_bytree=0.6,
    gamma=0.1,
    learning_rate=0.08,
    max_depth=6,
    subsample=0.4
)

In [64]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [66]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

x = pd.DataFrame(x)
y = pd.DataFrame(y)

for fold, (train_idx, val_idx) in enumerate(kf.split(x)):
    print(f"Fold {fold + 1}/{num_folds}")
    
    x_train, x_val = x.iloc[train_idx], x.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    y_train = y_train.values.reshape(-1, 1)
    y_val = y_val.values.reshape(-1, 1)
    
    model = XGBRFRegressor(**best_params)
    
    model.fit(x_train, y_train, eval_set=[(x_val, y_val)],
              verbose=50,
              eval_metric='mae')
    
    val_preds = model.predict(x_val)
    
    nmae_value = NMAE(y_val, val_preds)
    print(f"NMAE: {nmae_value}")

Fold 1/5


C:\Users\kt826\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\kt826\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg__estimator__colsample_bytree", "reg__estimator__gamma", "reg__estimator__learning_rate", "reg__estimator__max_depth", "reg__estimator__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mae:31.96050
NMAE: 0.5989855933042629
Fold 2/5


C:\Users\kt826\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg__estimator__colsample_bytree", "reg__estimator__gamma", "reg__estimator__learning_rate", "reg__estimator__max_depth", "reg__estimator__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mae:30.19956
NMAE: 0.6397882336818002
Fold 3/5


...

[0]	validation_0-mae:29.73211
NMAE: 0.6241786453726081
Fold 4/5


C:\Users\kt826\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg__estimator__colsample_bytree", "reg__estimator__gamma", "reg__estimator__learning_rate", "reg__estimator__max_depth", "reg__estimator__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mae:30.86283
NMAE: 0.6285823214516019
Fold 5/5


C:\Users\kt826\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg__estimator__colsample_bytree", "reg__estimator__gamma", "reg__estimator__learning_rate", "reg__estimator__max_depth", "reg__estimator__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mae:29.27709
NMAE: 0.5964461407923899


In [67]:
model.fit(x, y)

C:\Users\kt826\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [10:44:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg__estimator__colsample_bytree", "reg__estimator__gamma", "reg__estimator__learning_rate", "reg__estimator__max_depth", "reg__estimator__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg__estimator__colsample_bytree=0.6, ...)

In [58]:
xgb_model.fit(x_train, y_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=0.6, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=0.1,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.08, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=6, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, ...)

In [68]:
test = pd.read_csv("C:/Users/kt826/Desktop/test월별통계량.csv")
test

,ID,수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,2022-09-05 새순,...,11월_새순_var,9월_엽록소_mean,9월_엽록소_std,9월_엽록소_var,10월_엽록소_mean,10월_엽록소_std,10월_엽록소_var,11월_엽록소_mean,11월_엽록소_std,11월_엽록소_var
0,TEST_0000,231.0,219.0,302.0,260.5,4.3,4.2,4.2,4.1,4.1,...,0.163333,76.326280,14.439129,208.488445,76.036152,1.098762,1.207278,72.731341,0.877094,0.769294
1,TEST_0001,206.0,324.0,346.0,335.0,2.5,2.5,2.5,2.5,2.5,...,0.033651,71.549382,13.425141,180.234409,71.585544,0.686975,0.471934,68.870045,0.748416,0.560127
2,TEST_0002,282.0,373.0,404.0,388.5,4.7,4.7,4.6,4.6,4.5,...,0.193056,78.122212,14.775198,218.306463,77.683568,0.877720,0.770393,74.952479,0.919768,0.845974
3,TEST_0003,234.0,337.0,357.0,347.0,3.6,3.6,3.6,3.6,3.5,...,0.101640,77.186699,14.604025,213.277548,76.772764,0.848313,0.719635,74.053836,0.816307,0.666356
4,TEST_0004,222.0,329.0,347.0,338.0,2.7,2.7,2.7,2.7,2.6,...,0.036548,76.372353,14.321641,205.109388,75.756466,0.969093,0.939141,72.728466,0.762630,0.581604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,TEST_2203,217.0,265.0,288.0,276.5,2.5,2.5,2.4,2.4,2.4,...,0.029881,74.510382,13.984101,195.555082,74.318749,0.767538,0.589114,71.255441,0.947884,0.898485
2204,TEST_2204,182.0,191.0,262.0,226.5,4.1,4.1,4.1,4.0,4.0,...,0.141521,70.692327,13.376645,178.934629,70.289326,0.965549,0.932285,67.249234,0.922920,0.851781
2205,TEST_2205,254.0,280.0,332.0,306.0,3.8,3.8,3.8,3.7,3.7,...,0.089418,67.280967,12.620638,159.280511,66.678515,0.868139,0.753665,63.821761,0.899383,0.808891
2206,TEST_2206,257.0,285.0,297.0,291.0,4.5,4.4,4.4,4.3,4.3,...,0.176243,68.919126,13.037170,169.967803,68.274804,0.923429,0.852721,64.943420,0.917332,0.841498


In [69]:
pred = model.predict(test.iloc[:,1:])

In [70]:
submit = pd.read_csv("C:/Users/kt826/Desktop/open/sample_submission.csv")

In [71]:
submit['착과량(int)'] = pred

In [72]:
submit.to_csv('xgbrfkfold_submit.csv', index=False)